In [1]:
from preprocessing import get_features_and_target
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras import regularizers

# LSTM regression on returns

# Preprocessing 

In [37]:
symbol = "BTC/USDT"
feature_lags = []
valid_split = 2000
train_length = 48
batch_size = 32
seed = 44

df = get_features_and_target(
    symbol,
    days_to_forecast=1,
    feature_lags=feature_lags,
    model_type="reg",
)


symbol = symbol.replace("/", ":")

X = df.drop(columns=f"{symbol}_target")

y = df[f"{symbol}_target"].copy()

X_train, X_valid, y_train, y_valid = (
    X[:-valid_split],
    X[-valid_split:],
    y[:-valid_split],
    y[-valid_split:],
)


#### For Stateful LSTM
# Function to adjust dataset
def adjust_dataset(X, y, train_length, batch_size):
    total_sequences = len(X) - train_length + 1
    complete_batches = total_sequences // batch_size
    keep_sequences = complete_batches * batch_size
    keep_data_points = keep_sequences + train_length - 18

    X_adjusted = X.iloc[:keep_data_points]
    y_adjusted = y.iloc[train_length - 1 : keep_data_points]

    return X_adjusted, y_adjusted


# Adjusting training dataset
X_train_adjusted, y_train_adjusted = adjust_dataset(
    X_train, y_train, train_length, batch_size
)

# Adjusting validation dataset
X_valid_adjusted, y_valid_adjusted = adjust_dataset(
    X_valid, y_valid, train_length, batch_size
)

# Now, create the datasets
train_df_stateful = tf.keras.utils.timeseries_dataset_from_array(
    X_train_adjusted,
    targets=y_train_adjusted[(train_length - 1) :],
    sequence_length=train_length,
    batch_size=batch_size,
    shuffle=False,
    seed=seed,
)

valid_df_stateful = tf.keras.utils.timeseries_dataset_from_array(
    X_valid_adjusted,
    targets=y_valid_adjusted[(train_length - 1) :],
    sequence_length=train_length,
    batch_size=batch_size,
    shuffle=False,
    seed=seed,
)

# Check
# Initialize variables to hold the first batch's data
# Get the first batch from the dataset
for X_batch, y_batch in train_df_stateful.take(1):
    # Extract the first sample from the batch
    first_sample_features = X_batch[0][-1].numpy().tolist()
    first_sample_target = y_batch[0]

    # Print the feature vector and its corresponding target
    print("Feature Vector (First Sample):", first_sample_features)
    print("Target (First Sample):", first_sample_target)
    break  # Exit after processing the first batch

X_train_adjusted.iloc[47], y_train_adjusted[47]

Feature Vector (First Sample): [4320.661111111117, 4313.296499999998, 4313.8142000000025, 4139.704399999998, 48.092587697298235, 4376.440449600143, 4250.152550399852, 17.075320058893993, -2.8431784957280275, 54.29219000000007, 4342.780000000001, 27.8199457847807, 24.468002066163535, 0.0, 42.85714285714286]
Target (First Sample): tf.Tensor(0.026239868113751896, shape=(), dtype=float64)


/var/folders/cd/xc6lw60d5711s9k_4hjp_dwc0000gn/T/ipykernel_8564/1950526221.py:86: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  X_train_adjusted.iloc[47], y_train_adjusted[47]


(BTC:USDT_sma_9                  4320.661111
 BTC:USDT_sma_20                 4313.296500
 BTC:USDT_sma_50                 4313.814200
 BTC:USDT_sma_200                4139.704400
 BTC:USDT_rsi                      48.092588
 BTC:USDT_bollinger_up           4376.440450
 BTC:USDT_bollinger_down         4250.152550
 BTC:USDT_adx                      17.075320
 BTC:USDT_macd_diff                -2.843178
 BTC:USDT_obv                      54.292190
 BTC:USDT_ichimoku_conversion    4342.780000
 BTC:USDT_stochastic_k             27.819946
 BTC:USDT_stochastic_d             24.468002
 BTC:USDT_aroon_up                  0.000000
 BTC:USDT_aroon_down               42.857143
 Name: 2017-08-27 10:00:00, dtype: float64,
 0.026239868113751896)

# Model Regression

In [9]:
tf.keras.backend.clear_session()

# Preperation of layers
conv1 = tf.keras.layers.Conv1D(
    filters=60,
    kernel_size=3,
    strides=1,
    padding="causal",
    activation="relu",
    input_shape=[None, len(df.columns) - 1],
)

# Bidirectional LSTM tf.keras.layers
lstm1 = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(30, return_sequences=True, stateful=False)
)
lstm2 = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(30, return_sequences=False, stateful=True)
)
norm_layer = tf.keras.layers.Normalization()
norm_layer.adapt(X_train)

# Model
inputs = tf.keras.layers.Input(batch_size=32, shape=(None, len(df.columns) - 1))
x = norm_layer(inputs)
x = conv1(x)
x = lstm1(x)

x = lstm2(x)
x = tf.keras.layers.Dropout(0.2)(x)  # Increased dropout
x = tf.keras.layers.BatchNormalization()(x)  # Batch Normalization
x = tf.keras.layers.Dense(
    30,
    activation="relu",
    kernel_initializer="he_normal",
    kernel_regularizer=regularizers.l2(0.01),
)(
    x
)  # L2 regularization on Dense layer
outputs = tf.keras.layers.Dense(1)(x)
# Optimizers and losses
SGD_optimizer = tf.keras.optimizers.legacy.SGD(
    learning_rate=0.001, momentum=0.9, nesterov=True
)
adam_opt = tf.keras.optimizers.AdamW(learning_rate=0.001)

huber_loss = tf.keras.losses.Huber()
mse_loss = "mse"

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer="adam", loss=mse_loss, metrics=["mae", "RootMeanSquaredError"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(32, None, 15)]          0         
                                                                 
 normalization (Normalizati  (32, None, 15)            31        
 on)                                                             
                                                                 
 conv1d (Conv1D)             (32, None, 60)            2760      
                                                                 
 bidirectional (Bidirection  (32, None, 60)            21840     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (32, 60)                  21840     
 onal)                                                           
                                                             

In [10]:
# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=3, monitor="val_mae", restore_best_weights=True
)
tensorboard = tf.keras.callbacks.TensorBoard("TB_regression_logs")
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_mae",
    factor=0.5,  # Reduce learning rate by half
    patience=1,  # Number of epochs with no improvement
    min_lr=0.0001,  # Minimum learning rate
)
model.fit(
    train_df_stateful,
    validation_data=valid_df_stateful,
    callbacks=[early_stopping, tensorboard, reduce_lr],
    epochs=50,
)

Epoch 1/50
1537/1537 [==============================] - 69s 42ms/step - loss: 0.2411 - mae: 0.0827 - root_mean_squared_error: 0.2079 - val_loss: 0.0153 - val_mae: 0.0242 - val_root_mean_squared_error: 0.0323 - lr: 0.0010
Epoch 2/50
1537/1537 [==============================] - 62s 40ms/step - loss: 0.0046 - mae: 0.0303 - root_mean_squared_error: 0.0442 - val_loss: 5.2935e-04 - val_mae: 0.0133 - val_root_mean_squared_error: 0.0222 - lr: 0.0010
Epoch 3/50
1537/1537 [==============================] - 62s 41ms/step - loss: 0.0018 - mae: 0.0283 - root_mean_squared_error: 0.0422 - val_loss: 4.7535e-04 - val_mae: 0.0131 - val_root_mean_squared_error: 0.0218 - lr: 0.0010
Epoch 4/50
1537/1537 [==============================] - 63s 41ms/step - loss: 0.0017 - mae: 0.0274 - root_mean_squared_error: 0.0411 - val_loss: 4.6962e-04 - val_mae: 0.0131 - val_root_mean_squared_error: 0.0217 - lr: 0.0010
Epoch 5/50
1537/1537 [==============================] - 63s 41ms/step - loss: 0.0017 - mae: 0.0273 - roo

In [11]:
model.save("last_regression_model.keras")  # Save regression the model

# Classification model

# Preprocessing

In [2]:
symbol = "BTC/USDT"
feature_lags = []
valid_split = 2000
train_length = 48
batch_size = 32
seed = 44

df = get_features_and_target(
    symbol,
    days_to_forecast=1,
    feature_lags=feature_lags,
    model_type="class",
)

symbol = symbol.replace("/", ":")

X = df.drop(columns=f"{symbol}_target")

y = df[f"{symbol}_target"].copy()

X_train, X_valid, y_train, y_valid = (
    X[:-valid_split],
    X[-valid_split:],
    y[:-valid_split],
    y[-valid_split:],
)

train_df = tf.keras.utils.timeseries_dataset_from_array(
    X_train,
    targets=y_train[train_length:],
    sequence_length=train_length,
    batch_size=batch_size,
    shuffle=True,  # Shaffles the sequences, not within sequences
    seed=seed,
)

valid_df = tf.keras.utils.timeseries_dataset_from_array(
    X_valid,
    targets=y_valid[train_length:],
    sequence_length=train_length,
    batch_size=batch_size,
    shuffle=True,
    seed=seed,
)

for X_batch, y_batch in train_df.take(1):
    print("Train batch X shape:", X_batch.shape)
    print("Train batch y shape:", y_batch.shape)  # starts from y.iloc[48]

y.value_counts()

Train batch X shape: (32, 48, 15)
Train batch y shape: (32,)


2023-11-20 22:42:02.652569: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2023-11-20 22:42:02.652591: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-11-20 22:42:02.652599: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-11-20 22:42:02.652627: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-20 22:42:02.652648: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


BTC:USDT_target
0.0    35421
2.0     8564
1.0     7320
Name: count, dtype: int64

In [3]:
tf.keras.backend.clear_session()

# Preperation of layers
conv1 = tf.keras.layers.Conv1D(
    filters=60,
    kernel_size=3,
    strides=1,
    padding="causal",
    activation="relu",
    input_shape=[None, len(df.columns) - 1],
)

# Bidirectional LSTM tf.keras.layers
lstm1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(30, return_sequences=True))
# lstm2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(30, return_sequences=False))
norm_layer = tf.keras.layers.Normalization()
norm_layer.adapt(X_train)

# Model
inputs = tf.keras.layers.Input(shape=(None, len(df.columns) - 1))
x = norm_layer(inputs)
x = conv1(x)
x = lstm1(x)
# x = tf.keras.layers.Dropout(0.1)(x)
# x = lstm2(x)
# x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dense(30, activation="relu", kernel_initializer="he_normal")(x)
# x = tf.keras.layers.Dropout(0.1)(x)
outputs = tf.keras.layers.Dense(3, activation="softmax")(x)

# Optimizers and loss
SGD_optimizer = tf.keras.optimizers.legacy.SGD(
    learning_rate=0.001, momentum=0.9, nesterov=True
)
adam_opt = tf.keras.optimizers.AdamW(learning_rate=0.001)

categorical_loss = tf.keras.losses.SparseCategoricalCrossentropy()

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer=adam_opt,
    loss=categorical_loss,
    metrics=["accuracy", Precision(), Recall()],
)
model.summary()

2023-11-20 22:42:47.162845: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 15)]        0         
                                                                 
 normalization (Normalizati  (None, None, 15)          31        
 on)                                                             
                                                                 
 conv1d (Conv1D)             (None, None, 60)          2760      
                                                                 
 bidirectional (Bidirection  (None, None, 60)          21840     
 al)                                                             
                                                                 
 global_average_pooling1d (  (None, 60)                0         
 GlobalAveragePooling1D)                                         
                                                             

In [4]:
from sklearn.utils import class_weight


# class_weights = class_weight.compute_class_weight(
#     "balanced", classes=np.unique(y_train), y=y_train
# )
class_weights = {0: 0.48, 1: 2.26, 2: 1.95}

# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=3, monitor="accuracy", restore_best_weights=True
)

tensorboard = tf.keras.callbacks.TensorBoard("TB_classification_logs")
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="accuracy",
    factor=0.5,  # Reduce learning rate by half
    patience=1,  # Number of epochs with no improvement
    min_lr=0.0001,  # Minimum learning rate
)
model.fit(
    train_df,
    validation_data=valid_df,
    callbacks=[early_stopping, tensorboard, reduce_lr],
    epochs=50,
    class_weight=class_weights,
)

Epoch 1/50


ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1131, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1225, in compute_metrics

In [ ]:
model.save("last_classification_model.keras")